You need the .gz files from https://sites.google.com/eng.ucsd.edu/ucsdbookgraph/home

In [6]:
import gzip
import json
import re
import os
import sys
import numpy as np
import pandas as pd

# to delete all unnecessary columns and select all english books of "goodreads_books.json.gz" 
lang = ['en', 'en-CA', 'en-US', 'en-GB', 'eng']
toDel = ['country_code', 'is_ebook', 'kindle_asin',  'link', 'url','publication_day', 'publication_month', 'num_pages', 'popular_shelves']

def load_data(file_name, start, end):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            if count < start:
                count += 1
                continue
            elif count >= end:
                break
            else:
                d = json.loads(l)
                if d['language_code'] in lang:
                    for element in toDel:
                        d.pop(element)
                    data.append(d)
                count += 1
        print(len(data))
        return data
DIR = './'

In [7]:
# books = load_data(os.path.join(DIR, 'goodreads_books.json.gz'),0,500000)
# books = load_data(os.path.join(DIR, 'goodreads_books.json.gz'),500000,1000000)
# books = load_data(os.path.join(DIR, 'goodreads_books.json.gz'),1000000,1500000)
# books = load_data(os.path.join(DIR, 'goodreads_books.json.gz'),1500000,2000000)
# books = load_data(os.path.join(DIR, 'goodreads_books.json.gz'),2000000, 2370000)

131560


In [ ]:
with open('goodreadsBooks.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(books, ensure_ascii=False))

In [ ]:
def load_all_data(file_name):
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            data.append(d)
            
        print(len(data))
        return data
DIR = './'

In [ ]:
genre = load_all_data(os.path.join(DIR, 'goodreads_book_genres_initial.json.gz'))
with open('goodreadsGenre.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(genre, ensure_ascii=False))


In [ ]:
author = load_all_data(os.path.join(DIR, 'goodreads_book_authors.json.gz'))
with open('goodreadsAuthors.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(author, ensure_ascii=False))

In [ ]:
toDel = ['review_text', 'date_added', 'date_updated', 'read_at', 'started_at', 'n_votes', 'n_comments', 'review_id']

def load_data(file_name, start, end):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            if count < start:
                count += 1
                continue
            elif count >= end:
                break
            else:
                d = json.loads(l)
                for element in toDel:
                    d.pop(element)
                data.append(d)
                count += 1
        print(len(data))
        return data
DIR = './'

review = load_data(os.path.join(DIR, 'goodreads_reviews_dedup.json.gz'),12000000, 15000000)

In [ ]:
with open('goodreadsReview4.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(review, ensure_ascii=False))